In [1]:
# Install all required libraries — only need to run this once ever
%pip install pandas numpy scikit-learn xgboost shap matplotlib seaborn streamlit

print("✅ Done!")

Note: you may need to restart the kernel to use updated packages.
✅ Done!



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 2. Import Libraries
import pandas as pd
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

sns.set_style('whitegrid')
print("✅ Libraries loaded!")

✅ Libraries loaded!


In [3]:
# 3. Load CSV 
#skip row index 1 because it's a metadata tag row, not real data
df = pd.read_csv('data/wfp_food_prices_lka.csv', skiprows=[1])

print(f"Rows: {len(df):,}")
print(f"Columns: {df.columns.tolist()}")
df.head(3)

Rows: 22,588
Columns: ['date', 'admin1', 'admin2', 'market', 'market_id', 'latitude', 'longitude', 'category', 'commodity', 'commodity_id', 'unit', 'priceflag', 'pricetype', 'currency', 'price', 'usdprice']


,date,admin1,admin2,market,market_id,latitude,longitude,category,commodity,commodity_id,unit,priceflag,pricetype,currency,price,usdprice
0,2004-01-15,Western,Colombo,Colombo City,368,6.93,79.85,cereals and tubers,Rice (red nadu),157,KG,actual,Retail,LKR,32.71,0.33
1,2004-02-15,Western,Colombo,Colombo City,368,6.93,79.85,cereals and tubers,Rice (red nadu),157,KG,actual,Retail,LKR,33.62,0.34
2,2004-03-15,Western,Colombo,Colombo City,368,6.93,79.85,cereals and tubers,Rice (red nadu),157,KG,actual,Retail,LKR,32.92,0.34


In [4]:
# 4 — Explore the Data
# See what provinces, commodities and date range we have
print("Provinces:")
print(df['admin1'].dropna().unique())

print("\nCategories:")
print(df['category'].dropna().unique())

print(f"\nDate range: {df['date'].min()} → {df['date'].max()}")
print(f"\nPrice stats (LKR):")
df['price'].describe()

Provinces:
['Western' 'Eastern' 'Northern' 'Central' 'North Central' 'North Western'
 'Sabaragamuwa' 'Southern' 'Uva']

Categories:
['cereals and tubers' 'miscellaneous food' 'non-food' 'pulses and nuts'
 'vegetables and fruits' 'meat, fish and eggs' 'oil and fats']

Date range: 2004-01-15 → 2025-09-15

Price stats (LKR):


count    22588.000000
mean       601.125075
std        622.551318
min         16.000000
25%        182.000000
50%        336.000000
75%        864.317500
max       3735.420000
Name: price, dtype: float64

In [5]:
# 5 — Clean the Data
# Keep only columns we need
df = df[['date', 'admin1', 'market', 'category', 'commodity', 'unit', 'price']].copy()

# Drop rows with missing prices
before = len(df)
df.dropna(subset=['price'], inplace=True)
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df.dropna(subset=['price'], inplace=True)
print(f"Removed {before - len(df):,} rows with missing prices → {len(df):,} remain")

# Remove extreme outliers — prices above 99th percentile are likely errors
upper = df['price'].quantile(0.99)
before = len(df)
df = df[df['price'] <= upper]
print(f"Removed {before - len(df):,} outliers (above LKR {upper:.2f})")

print(f"\n✅ Clean dataset: {len(df):,} rows")

Removed 0 rows with missing prices → 22,588 remain
Removed 224 outliers (above LKR 2550.00)

✅ Clean dataset: 22,364 rows


In [6]:
# 6 — Extract Date Features
df['date']  = pd.to_datetime(df['date'])
df['year']  = df['date'].dt.year
df['month'] = df['date'].dt.month

print(f"Year range: {df['year'].min()} – {df['year'].max()}")
print(f"Sample:")
df[['date', 'year', 'month', 'price']].head(5)

Year range: 2004 – 2025
Sample:


,date,year,month,price
0,2004-01-15,2004,1,32.71
1,2004-02-15,2004,2,33.62
2,2004-03-15,2004,3,32.92
3,2004-03-15,2004,3,26.63
4,2004-04-15,2004,4,32.98


In [7]:
# Prepare data for encoding
# Just sort by date for consistency
df = df.sort_values('date')

print("✅ Data sorted!")

✅ Data sorted!


In [8]:
# 8 — Label Encode Categorical Columns
# LabelEncoder converts each unique text value to a unique integer
# e.g. "Western" → 0, "Eastern" → 1, "Northern" → 2

cat_cols = ['admin1', 'market', 'category', 'commodity', 'unit']
encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    df[col + '_enc'] = le.fit_transform(df[col].astype(str))
    encoders[col] = le  # save so we can reverse it in the frontend
    print(f"{col}: {len(le.classes_)} unique values")

print("\n✅ Encoding complete!")
print(df[['admin1', 'admin1_enc', 'commodity', 'commodity_enc', 'price']].head(5))

admin1: 10 unique values
market: 44 unique values
category: 7 unique values
commodity: 42 unique values
unit: 4 unique values

✅ Encoding complete!
    admin1  admin1_enc        commodity  commodity_enc  price
0  Western           8  Rice (red nadu)             35  32.71
1  Western           8  Rice (red nadu)             35  33.62
2  Western           8  Rice (red nadu)             35  32.92
3  Western           8      Wheat flour             41  26.63
4  Western           8  Rice (red nadu)             35  32.98


In [9]:
# 9 — Save Processed Data & Encoders
# Save cleaned data for the training notebook
df.to_csv('outputs/processed_data.csv', index=False)

# Save encoders so the frontend can decode user selections
with open('models/encoders.pkl', 'wb') as f:
    pickle.dump(encoders, f)

print("✅ Saved: outputs/processed_data.csv")
print("✅ Saved: models/encoders.pkl")
print(f"\nFinal dataset shape: {df.shape}")
print(f"Price range: LKR {df['price'].min():.2f} – {df['price'].max():.2f}")
print(f"\nFeatures in dataset:")
print(df.columns.tolist())

✅ Saved: outputs/processed_data.csv
✅ Saved: models/encoders.pkl

Final dataset shape: (22364, 14)
Price range: LKR 16.00 – 2550.00

Features in dataset:
['date', 'admin1', 'market', 'category', 'commodity', 'unit', 'price', 'year', 'month', 'admin1_enc', 'market_enc', 'category_enc', 'commodity_enc', 'unit_enc']
